In [16]:
import numpy as np
import tensorflow as tf
import os, sys
from tensorflow.keras.layers import Input

# find root directory and add to path
root_index = os.getcwd().rfind('notebooks')
root_directory = os.path.dirname(os.getcwd()[:root_index])
sys.path.append(root_directory)
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from MotorNet.plants import RigidTendonArm
from MotorNet.plants.muscles import RigidTendonHillMuscle, RigidTendonHillMuscleThelen
from MotorNet.nets.layers import GRUController
from MotorNet.tasks.tasks import TaskLoadProbability, TaskStaticTarget, TaskDelayedReach, TaskStaticTargetWithPerturbations, TaskDelayedMultiReach
from MotorNet.nets.custommodels import MotorNetModel

%load_ext autoreload
%autoreload 2
print('tensorflow version: ' + tf.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
tensorflow version: 2.4.1


In [17]:
save_folder = '/home/jonathan/Desktop/MotorNetModels/LoadProb/'
save_name = 'LoadProb'
# specify number of models in batch
models_in_batch = 4

if not os.path.exists(save_folder):
    os.mkdir(save_folder)

for i in range(models_in_batch):
    # Create model
    arm = RigidTendonArm(muscle_type=RigidTendonHillMuscleThelen(), timestep=0.01,
                         proprioceptive_delay=0.04, visual_delay=0.09,
                         excitation_noise_sd=0.0001)
    visual_feedback_noise = 0.01
    proprio_feedback_noise = 0.001
    hidden_noise = 0.00001
    cell = GRUController(plant=arm, n_units=300, kernel_regularizer=1e-6,
                         recurrent_regularizer=2e-5, name='cell',
                         proprioceptive_noise_sd=proprio_feedback_noise, visual_noise_sd=visual_feedback_noise,
                         hidden_noise_sd=hidden_noise)

    delay_range = [200, 1200]
    target_time_range = [200, 400]
    if i < 2:
        do_recompute_targets = False
    else:
        do_recompute_targets = True
    cartesian_loss = 1
    muscle_loss = 20
    task = TaskLoadProbability(cell, initial_joint_state=np.deg2rad([60., 80., 0., 0.]),
                               delay_range=delay_range,
                               target_time_range=target_time_range,
                               do_recompute_targets=do_recompute_targets,
                               cartesian_loss=cartesian_loss,
                               muscle_loss=muscle_loss
                               )

    # declare inputs
    inputs = Input((None, task.get_input_dim()), name='inputs')
    state0 = [Input((arm.state_dim, ), name='joint0'),
              Input((arm.state_dim, ), name='cartesian0'),
              Input((arm.muscle_state_dim, arm.n_muscles, ), name='muscle0'),
              Input((arm.geometry_state_dim, arm.n_muscles, ), name='geometry0'),
              Input((arm.n_muscles*2, arm.proprioceptive_delay, ), name='proprio_feedback0'),
              Input((arm.space_dim, arm.visual_delay, ), name='visual_feedback0')]
    state0.extend([Input((n, ), name='gru' + str(k) + '_hidden0') for k, n in enumerate(cell.n_units)])

    # wrap cell in an RNN layer
    states_out = tf.keras.layers.RNN(cell=cell, return_sequences=True, name='RNN')(inputs, initial_state=state0)
    control_rnn = MotorNetModel(inputs=[inputs, state0], outputs=states_out, name='controller', task=task)

    batch_size = 32
    n_t = int(2.0 / arm.dt)
    control_rnn.task.set_training_params(batch_size=batch_size, n_timesteps=n_t, iterations=15000)

    # save model config
    control_rnn.save_model(save_folder + save_name + '_' + str(i))